<a href="https://colab.research.google.com/github/tasjapr/diploma/blob/master/my_try.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!git clone https://github.com/tensorflow/models.git

In [0]:
import os
os.chdir('models/')

Cloning into 'generate_tfrecord.py'...
fatal: repository 'https://github.com/datitran/raccoon_dataset/blob/master/generate_tfrecord.py/' not found
